In [6]:
import numpy       as np
import matplotlib.pyplot as plt
import numpy       as np
import matplotlib as mpl
import pandas as pd
import glob 
import os

In [28]:
def Weigfun( Tlag): 
    # WEIGFUN Summary of this function goes here
    #   Detailed explanation goes here
    nmax=int(np.ceil(Tlag))
    
    if nmax==1: 
        Weigths=float(1)
    else:
        Weigths=np.zeros(nmax)

        th=Tlag/2
        nh=int(np.floor(th))
        for i in range(0,nh): 
            Weigths[i]=(float(i+1)-0.5)/th        
        i=nh

        Weigths[i]=(1+(float(i+1)-1)/th)*(th-int(np.floor(th)))/2+(1+(Tlag-float(i+1))/th)*(int(np.floor(th))+1-th)/2
        for i in range(nh+1, int(np.floor(Tlag))):
            Weigths[i]=(Tlag-float(i+1)+.5)/th

        if Tlag>int(np.floor(Tlag)):
            Weigths[int(np.floor(Tlag))]=(Tlag-int(np.floor(Tlag)))**2/(2*th)

        Weigths=Weigths/sum(Weigths)

    return(Weigths)
    # plot(Weigths)


In [29]:
def HBVMod(Par,forcing,Sin, hydrograph):
        #HBVpareto Calculates values of 3 objective functions for HBV model

    Imax = Par[0]
    Ce = Par[1]
    Sumax = Par[2]
    beta = Par[3]
    Pmax = Par[4]
    Tlag = Par[5]
    Kf = Par[6]
    Ks = Par[7]
    

    Prec = forcing['P'].values
    Qo = forcing['Q'].values
    Etp = forcing['Pev'].values


    tmax = len(Prec)
    
    # allocate Si, Su, Sf, Ss, Eidt, Eadt, Qtotdt
    
    Si = np.zeros(tmax)
    Su = np.zeros(tmax)
    Sf = np.zeros(tmax)
    Ss = np.zeros(tmax)
    Eidt = np.zeros(tmax)
    Eadt =  np.zeros(tmax)
    Qtotdt = np.zeros(tmax)
    Qs = np.zeros(tmax)
    Qf = np.zeros(tmax)
    
    # initialize Si, Su, Sf, Ss
    Si[0] = Sin[0]
    Su[0] = Sin[1]
    Sf[0] = Sin[2]
    Ss[0] = Sin[3]

    dt = 1

    #
    # Model 1 SOF1
    for i in range(0, tmax):
        Pdt = Prec[i] * dt
        Epdt = Etp[i] * dt
        
        # Interception Reservoir
        if Pdt > 0:
            Si[i] = Si[i] + Pdt
            Pedt = np.maximum(0, (Si[i] - Imax) / dt)
            Si[i] = Si[i] - Pedt
            Eidt[i] = 0
        else:
        # Evaporation only when there is no rainfall
            Pedt = np.maximum(0, (Si[i] - Imax) / dt) #is zero, because of no rainfall
            Eidt[i] = np.minimum(Epdt, Si[i] / dt)
            Si[i] = Si[i] - Pedt - Eidt[i]
        
        if i < tmax-1:
            Si[i+1] = Si[i]
        
        
        # Split Pe into Unsaturated Reservoir and Preferential reservoir
        if Pedt > 0:
            Cr = (Su[i] / Sumax) ** beta
            Qiudt = (1 - Cr) * Pedt # flux from Ir to Ur
            Su[i] = Su[i] + Qiudt
            Qufdt = Cr * Pedt #flux from Su to Sf
        else:
            Qufdt = 0
        
        # Transpiration
        Epdt = max(0, Epdt - Eidt[i])
        Eadt[i] = Epdt * (Su[i] / (Sumax * Ce))
        Eadt[i] = min(Su[i] / dt, Eadt[i])
        Su[i] = Su[i] - Eadt[i]
        
        # Percolation
        Qusdt = Pmax * (Su[i] / Sumax) * dt # Flux from Su to Ss
        Su[i] = Su[i] - Qusdt
        
        if i < tmax - 1:
            Su[i+1] = Su[i]
        
        # Fast Reservoir
        Sf[i] = Sf[i] + Qufdt
        Qfdt = dt * Kf * Sf[i]
        Sf[i] = Sf[i] - Qfdt
        if i < tmax-1:
            Sf[i+1] = Sf[i]
        
        # Slow Reservoir
        Ss[i] = Ss[i] + Qusdt
        Qsdt = dt * Ks * Ss[i]
        Ss[i] = Ss[i] - Qsdt
        if i < tmax-1:
            Ss[i+1] = Ss[i]
        
        Qtotdt[i] = Qsdt + Qfdt
        Qs[i] = Qsdt 
        Qf[i] = Qfdt 


    # Check Water Balance
    Sf = Si[-1] + Ss[-1] + Sf[-1] + Su[-1] #final storage
    Sin = sum(Sin) #initial storage
    WB = sum(Prec) - sum(Eidt) - sum(Eadt) - sum(Qtotdt) - Sf + Sin

    # Offset Q

    Weigths = Weigfun(Tlag)
    
    Qm = np.convolve(Qtotdt, Weigths)
    Qm = Qm[0:tmax]
    forcing['Qm'] = Qm
    
    # Calculate objective
    ind = np.where(Qo>=0)
    QoAv = np.mean(Qo[ind])
    ErrUp = np.sum((Qo - Qm) ** 2)
    ErrDo = np.sum((Qo - QoAv) ** 2)
    Obj = 1 - (ErrUp / ErrDo)

    if hydrograph == 'True':
    ## Plot
    # hour=1:tmax\
        plt.plot(range(0,len(Qo)),Qo)
        plt.plot(range(0,len(Qm)),Qm)
        plt.show()
    if hydrograph == 'TRUE':
    ## Plot
    # hour=1:tmax\
        fig, ax = plt.subplots(figsize=(12,8))
        forcing['Q'].plot(label='Observed', ax=ax)
        forcing['Qm'].plot(label='Model',  ax=ax)
        ax.set_xlabel('Time [days]')
        ax.set_ylabel('Runoff Q [$mmd^{-1}$]')

        ax.legend()
        
    return(Obj)


    # leg['Qobs','Qmod']

In [30]:
path = os.getcwd()
home_path = os.path.dirname(path)
data_folder = f'{home_path}\\Data'
files = glob.glob(f"{data_folder}\\*.parquet")
df_data = pd.read_parquet(files[0])
data_folder

'C:\\Users\\anne-\\OneDrive - Delft University of Technology\\Documenten\\Environmental Engineering MSc\\ENVM1502-Catchment-model\\Data'

In [33]:
          #      Imax Ce Sumax beta Pmax   Tlag   Kf  Ks
ParMinn = np.array([0.01,   0.2,  40,    .5,   .001,   0.001,     .01,  .0001])
ParMaxn = np.array([8,    1,  800,   4,    .3,     10,    .1,   .01])
Sin = np.array([0,  100,  0,  5  ])




# GLUE
nmax = 50
A = np.zeros((nmax,9))
n_feasible = 0

for n in range(1,nmax): 
    Rnum = np.random.random(8)  #generate a vector of random number
    Par = Rnum * (ParMaxn - ParMinn) + ParMinn # calculate the random parameter set
    Obj = HBVMod(Par, df_data, Sin, hydrograph='FALSE') #call the model

    if Obj>.4:
        A[n_feasible,0:8]= Par
        A[n_feasible,8] = Obj
        n_feasible = n_feasible + 1
    


np.savetxt(f'{data_folder}\\MC2.txt',A[0:n_feasible,:], delimiter =',')


#find the optimum

Opt_obj = A.argmax(axis=0)[8]
print(Opt_obj)

#find the optimal parameter set
OptPar = A[Opt_obj, 0:8]


Obj=HBVMod(OptPar,df_data,Sin, hydrograph='True')

0


C:\Users\anne-\AppData\Local\Temp\ipykernel_17564\339717717.py:74: RuntimeWarning: divide by zero encountered in double_scalars
  Eadt[i] = Epdt * (Su[i] / (Sumax * Ce))
C:\Users\anne-\AppData\Local\Temp\ipykernel_17564\339717717.py:74: RuntimeWarning: invalid value encountered in double_scalars
  Eadt[i] = Epdt * (Su[i] / (Sumax * Ce))
C:\Users\anne-\AppData\Local\Temp\ipykernel_17564\339717717.py:79: RuntimeWarning: invalid value encountered in double_scalars
  Qusdt = Pmax * (Su[i] / Sumax) * dt # Flux from Su to Ss
C:\Users\anne-\AppData\Local\Temp\ipykernel_17564\4116089061.py:17: RuntimeWarning: invalid value encountered in double_scalars
  Weigths[i]=(1+(float(i+1)-1)/th)*(th-int(np.floor(th)))/2+(1+(Tlag-float(i+1))/th)*(int(np.floor(th))+1-th)/2
C:\Users\anne-\AppData\Local\Temp\ipykernel_17564\4116089061.py:17: RuntimeWarning: divide by zero encountered in double_scalars
  Weigths[i]=(1+(float(i+1)-1)/th)*(th-int(np.floor(th)))/2+(1+(Tlag-float(i+1))/th)*(int(np.floor(th))+1-

IndexError: index 0 is out of bounds for axis 0 with size 0

In [ ]:
Sin= np.array([0,  100,  0,  5  ])


A=np.genfromtxt('MC2.txt',  dtype=float, autostrip=True, delimiter=',')


#find the optimum
Opt_obj = A.argmax(axis=0)[8]
print(Opt_obj)

#find the optimal parameter set
OptPar = A[Opt_obj, 0:8]


plt.figure(1)
Obj = HBVMod(OptPar,df_data,Sin, hydrograph='TRUE')

fig, ax = plt.subplots(4,2, figsize=(10,10))


ax[0,0].plot(A[:,0],A[:,8],'.')
ax[0,0].set_xlabel('I$_{max}$', fontsize=15)
ax[0,0].set_ylabel('N (-)', fontsize=15)

plt.subplot(422)
ax[0,1].plot(A[:,1],A[:,8],'.')
ax[0,1].set_xlabel('C$_{e}$', fontsize=15)
ax[0,1].set_ylabel('N (-)', fontsize=15)

plt.subplot(423)
ax[1,0].plot(A[:,2],A[:,8],'.')
ax[1,0].set_xlabel('S$_{u,max}$', fontsize=15)
ax[1,0].set_ylabel('N (-)', fontsize=15), 

plt.subplot(424)
ax[1,1].plot(A[:,3],A[:,8],'.')
ax[1,1].set_xlabel('\beta', fontsize=15)
ax[1,1].set_ylabel('N (-)', fontsize=15)

plt.subplot(425)
ax[2,0].plot(A[:,4],A[:,8],'.')
ax[2,0].set_xlabel('P$_{max}$', fontsize=15)
ax[2,0].set_ylabel('N (-)', fontsize=15)

plt.subplot(426)
ax[2,1].plot(A[:,5],A[:,8],'.')
ax[2,1].set_xlabel('T$_{lag}$', fontsize=15)
ax[2,1].set_ylabel('N (-)', fontsize=15)

plt.subplot(427)
ax[3,0].plot(A[:,6],A[:,8],'.')
ax[3,0].set_xlabel('K$_{f}$', fontsize=15)
ax[3,0].set_ylabel('N (-)', fontsize=15)

plt.subplot(428)
ax[3,1].plot(A[:,7],A[:,8],'.')
ax[3,1].set_xlabel('K$_{s}$', fontsize=15)
ax[3,1].set_ylabel('N (-)', fontsize=15)

plt.tight_layout()
plt.show()



In [ ]:
np.random.random(8)